# Projeto 1 - Ciência dos Dados

Nome: _____

Nome: Ellen Coutinho Lião da Silva

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import re
import os

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\ellen\Downloads\cdados_p1\22-2a-cd-p1-grupo_alexiabp\notebooks


Carregando a base de dados com as notícias classificadas manualmente:

In [3]:
path = pathlib.Path(r"..\data")
filename = f'{path}\dados.xlsx'

In [4]:
train = pd.read_excel(filename, sheet_name = 'Treinamento')
train.head(5)

,Categoria,Titulo,Descrição,Data,Pagina,Target
0,financas,CORREÇÃO-Dólar caminha para 4ª alta semanal se...,(Corrige no 9º parágrafo para “outubro” no lug...,13/05/2022 12:04,71,1
1,financas,"Ouro fecha em alta, após quedas recentes, com ...",O contrato mais líquido do ouro fechou em alta...,03/05/2022 15:16,78,2
2,financas,"Bolsas da Europa fecham em queda, sem resoluçã...",As principais bolsas europeias fecharam em que...,10/03/2022 14:26,110,0
3,economia,"Aneel aprova reajuste médio de 15,77% nas tari...",A Agência Nacional de Energia Elétrica (Aneel)...,23/08/2022 11:21,57,0
4,financas,Dólar reduz queda com fortalecimento do DXY ap...,O dólar reduziu a queda intradia frente ao rea...,18/08/2022 10:33,12,1


In [5]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

C:\Users\ellen\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
C:\Users\ellen\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


,Categoria,Titulo,Descrição,Data,Pagina,Target
0,economia,Atividade econômica do País está acima do espe...,O diretor de Política Econômica do Banco Centr...,11/07/2022 12:57,212,0
1,economia,Tire suas dúvidas sobre a vacina contra a varí...,Um lote da vacina contra a varíola dos macacos...,17/08/2022 21:20,73,0
2,financas,Taxas futuras de juros recuam com ajustes apes...,As taxas de juros negociadas no mercado futuro...,15/07/2022 11:10,32,0
3,economia,Corte de Nova York permite que Harvey Weinstei...,Por Daniel Trotta (Reuters) – A mais alta cort...,25/08/2022 11:00,48,0
4,financas,"Bolsas de NY fecham em baixa, com guerra na Uc...",As bolsas de Nova York fecharam em baixa nesta...,03/03/2022 18:26,115,0


___
## Classificador automático


Faça aqui uma descrição do seu assunto e o que considerou como relevante ou não relevante na classificação das notícias (Target).

ESCREVA AQUI...

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

### Vamos agora fazer uma limpeza no título e na descrição de cada ntícia com a target = 0
Target 0 = neutro
Target 1 = positivo
Target 2 = negativo

In [6]:
def cleanup(text):
    #import string
    text = text.replace('[…]','')
    punctuation = '[!-.:?;\/–]' 
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

In [7]:
#Juntando Título e descrição do target 0
target_0 = test.loc[train['Target']==0,:]
string_0 = ''
for texto in target_0['Descrição']:
    novo_texto=cleanup(str(texto).lower())
    string_0+=''.join(novo_texto)
    
for titulo in target_0['Titulo']:
    novo_titulo=cleanup(str(titulo).lower())
    string_0+=''.join(novo_titulo)
    
    
lista_0 = string_0.split()

In [8]:
#Juntando Título e descrição do target 1
target_1 = test.loc[train['Target']==1,:]
string_1 = ''
for texto in target_1['Descrição']:
    novo_texto=cleanup(str(texto).lower())
    string_1+=''.join(novo_texto)
    
for titulo in target_1['Titulo']:
    novo_titulo=cleanup(str(titulo).lower())
    string_1+=''.join(novo_titulo)
    
    
lista_1 =string_1.split()

In [9]:
#Juntando Título e Descrição do Target 2
target_2 = test.loc[train['Target']==2,:]
string_2 = ''
for texto in target_2['Descrição']:
    novo_texto=cleanup(str(texto).lower())
    string_2+=''.join(novo_texto)
    
for titulo in target_2['Titulo']:
    novo_titulo=cleanup(str(titulo).lower())
    string_2+=''.join(novo_titulo)
    
    
lista_2 = string_2.split()

### Agora, faremos uma tabela de frequencias relativas e absolutas para cada stringzona! Queremos a frequência de cada palavra

### Cálculo da frequencia para targets 2

In [10]:
serie_0 = pd.Series(string_0.split())
tabela_0 = serie_0.value_counts(True)
tabela_0

de              0.057263
a               0.032068
o               0.031525
em              0.026040
do              0.023568
                  ...   
“caixa          0.000060
importante      0.000060
insatisfação    0.000060
escadas         0.000060
força’          0.000060
Length: 3854, dtype: float64

In [11]:
#Frequencia absoluta para Serie  0
tabela_abs_0 = serie_0.value_counts()
tabela_abs_0

de              950
a               532
o               523
em              432
do              391
               ... 
“caixa            1
importante        1
insatisfação      1
escadas           1
força’            1
Length: 3854, dtype: int64

### Cálculo da frequencia para targets 1 

In [12]:
serie_1 = pd.Series(string_1.split())
tabela_1 = serie_1.value_counts(True)
tabela_1

de         0.058670
o          0.033898
a          0.030639
em         0.029335
do         0.026076
             ...   
forte      0.000652
atraiu     0.000652
ativo      0.000652
assunto    0.000652
foco       0.000652
Length: 705, dtype: float64

In [13]:
#Frequencia absoluta para Serie  0
tabela_abs_1 = serie_1.value_counts()
tabela_abs_1

de         90
o          52
a          47
em         45
do         40
           ..
forte       1
atraiu      1
ativo       1
assunto     1
foco        1
Length: 705, dtype: int64

### Cálculo da frequencia para targets 2

In [14]:
serie_2 = pd.Series(string_2.split())
tabela_2 = serie_2.value_counts(True)
tabela_2

de            0.060514
a             0.037821
em            0.033283
o             0.026223
e             0.020171
                ...   
segue         0.000504
preocupado    0.000504
ritmo         0.000504
dirigentes    0.000504
processa      0.000504
Length: 857, dtype: float64

In [15]:
#Frequencia absoluta para Serie  2
tabela_abs_2 = serie_2.value_counts()
tabela_abs_2

de            120
a              75
em             66
o              52
e              40
             ... 
segue           1
preocupado      1
ritmo           1
dirigentes      1
processa        1
Length: 857, dtype: int64

Para fins de comparação, é necessário também unir todas as palavras das notícias em uma única string e montar as tabelas de frequências

In [17]:
todas = string_0+string_1+string_2
lista_todas=todas.split()
serie_todas = pd.Series(lista_todas)

tabela_todas = serie_todas.value_counts()
tabela_abs_todas = serie_todas.value_counts(True)
tabela_todas



de              1160
a                654
o                626
em               543
do               463
                ... 
relatórios         1
apontam            1
surpresas          1
pressionados       1
processa           1
Length: 4355, dtype: int64

## Montando o classificador

In [18]:
prob_neutra = len(serie_0)/len(serie_todas)
prob_positiva = len(serie_1)/len(serie_todas)
prob_negativa= len(serie_2)/len(serie_todas)

In [21]:
#P(palavra|target)
#noticia=cleanup(str(texto).lower())

test['Junção']=test['Titulo']+' '+test['Descrição']
i=0
for noticia in test['Junção']:
    #print(noticia)
    #print(' ')
    nova_noticia = str(noticia).lower()
    nova_noticia = cleanup(nova_noticia)
    
    #print(nova_noticia)
    test['Junção'][i]=nova_noticia
    i+=1
    
test

C:\Users\ellen\AppData\Local\Temp/ipykernel_2296/3004048147.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Junção'][i]=nova_noticia


,Categoria,Titulo,Descrição,Data,Pagina,Target,Junção
0,economia,Atividade econômica do País está acima do espe...,O diretor de Política Econômica do Banco Centr...,11/07/2022 12:57,212,0,atividade econômica do país está acima do espe...
1,economia,Tire suas dúvidas sobre a vacina contra a varí...,Um lote da vacina contra a varíola dos macacos...,17/08/2022 21:20,73,0,tire suas dúvidas sobre a vacina contra a varí...
2,financas,Taxas futuras de juros recuam com ajustes apes...,As taxas de juros negociadas no mercado futuro...,15/07/2022 11:10,32,0,taxas futuras de juros recuam com ajustes apes...
3,economia,Corte de Nova York permite que Harvey Weinstei...,Por Daniel Trotta (Reuters) – A mais alta cort...,25/08/2022 11:00,48,0,corte de nova york permite que harvey weinstei...
4,financas,"Bolsas de NY fecham em baixa, com guerra na Uc...",As bolsas de Nova York fecharam em baixa nesta...,03/03/2022 18:26,115,0,bolsas de ny fecham em baixa com guerra na ucr...
...,...,...,...,...,...,...,...
299,economia,Militares chineses enviarão tropas à Rússia pa...,PEQUIM (Reuters) – Tropas chinesas viajarão à ...,17/08/2022 07:46,77,0,militares chineses enviarão tropas à rússia pa...
300,economia,"O risco é a guerra, não a inflação, diz Zoltan...",A trajetória esperada para as taxas de juros n...,02/08/2022 15:31,129,0,o risco é a guerra não a inflação diz zoltan d...
301,economia,Homem inocentado após 20 anos preso por assass...,Por Jonathan Stempel NOVA YORK (Reuters) – Um ...,14/07/2022 20:28,194,0,homem inocentado após 20 anos preso por assass...
302,economia,EUA inclui intermediários chinês e indiano em ...,Os Estados Unidos impuseram nesta quinta-feira...,16/06/2022 13:36,295,0,eua inclui intermediários chinês e indiano em ...


In [24]:
probNeutra = []
probPos = []
probNeg = []

for noticia in test['Junção']:
    separacao = noticia.split()
    
    probJuncao_dado_Neutra = 1
    probJuncao_dado_Pos = 1
    probJuncao_dado_Neg = 1
    
    for palavra in separacao:
        #Partir daqui

atividade econômica do país está acima do esperado no 1º semestre diz guillen o diretor de política econômica do banco central bc diogo guillen disse nesta segundafeira 11 que a atividade econômica brasileira desempenhou em nível acima do esperado no primeiro semestre ele participou de um webinar organizado pelo credit suisse pela manhã de acordo com guillen o melhor desempenho envolve fatores transitórios como a normalização póspandemia e 
atividade
econômica
do
país
está
acima
do
esperado
no
1º
semestre
diz
guillen
o
diretor
de
política
econômica
do
banco
central
bc
diogo
guillen
disse
nesta
segundafeira
11
que
a
atividade
econômica
brasileira
desempenhou
em
nível
acima
do
esperado
no
primeiro
semestre
ele
participou
de
um
webinar
organizado
pelo
credit
suisse
pela
manhã
de
acordo
com
guillen
o
melhor
desempenho
envolve
fatores
transitórios
como
a
normalização
póspandemia
e
tire suas dúvidas sobre a vacina contra a varíola dos macacos um lote da vacina contra a varíola dos macacos de

bolsonaro
empresas enfrentam ações no exterior adaptar as empresas às questões ambientais sociais e de governança esg na sigla em inglês além de uma demanda da comunidade e de investidores tornouse necessidade em razão do risco legal às corporações o motivo é a tendência de tribunais da europa e dos estados unidos de reconhecer a jurisdição daqueles países para julgar ações 
empresas
enfrentam
ações
no
exterior
adaptar
as
empresas
às
questões
ambientais
sociais
e
de
governança
esg
na
sigla
em
inglês
além
de
uma
demanda
da
comunidade
e
de
investidores
tornouse
necessidade
em
razão
do
risco
legal
às
corporações
o
motivo
é
a
tendência
de
tribunais
da
europa
e
dos
estados
unidos
de
reconhecer
a
jurisdição
daqueles
países
para
julgar
ações
juros ficam estáveis com mercado ponderando câmbio treasuries e petróleo os juros terminaram a sessão regular praticamente estáveis com viés de baixa nos longos e de alta nos curtos a forte queda do dólar ante o real e o recuo do rendimento dos treasuries

chinês
fechou
em
forte
queda
quase
5
em
dalian
e
o
petróleo
vai
petróleo fecha em alta após sessão volátil com foco em china e payroll dos eua o petróleo encerrou em alta nesta sextafeira 7 em dia marcado pela volatilidade nos contratos futuros da commodity energética em meio ao relatório de empregos dos estados unidos o payroll de abril o óleo acentuou quedas nos minutos seguintes à divulgação dos números bem abaixo das expectativas de analistas mas retomou o avanço visto no 
petróleo
fecha
em
alta
após
sessão
volátil
com
foco
em
china
e
payroll
dos
eua
o
petróleo
encerrou
em
alta
nesta
sextafeira
7
em
dia
marcado
pela
volatilidade
nos
contratos
futuros
da
commodity
energética
em
meio
ao
relatório
de
empregos
dos
estados
unidos
o
payroll
de
abril
o
óleo
acentuou
quedas
nos
minutos
seguintes
à
divulgação
dos
números
bem
abaixo
das
expectativas
de
analistas
mas
retomou
o
avanço
visto
no
tóquio ultrapassa 30000 casos diários de covid pela primeira vez na pandemia tóquio reuters  novos ca

pregão
consecutivo
de
queda
rompeu
o
piso
de
r
475
e
encerrou
a
semana
marcada
por
retorno
mais
forte
do
apetite
ao
risco
no
exterior
com
desvalorização
de
279
uma
vez
mais
o
mercado
doméstico
de
câmbio
acompanhou
a
onda
de
enfraquecimento
global
da
moeda
guedes defende ‘pec kamikaze’ e a investidores diz que ‘fiscal está forte’ no dia seguinte à aprovação da proposta de emenda à constituição pec que amplia e cria novos benefícios sociais a três meses das eleições o ministro da economia paulo guedes chamou nesta quinta 14 os críticos das medidas de “politicamente enviesados” e tentou tranquilizar os investidores ao dizer que não haverá desequilíbrio fiscal com o 
guedes
defende
‘pec
kamikaze’
e
a
investidores
diz
que
‘fiscal
está
forte’
no
dia
seguinte
à
aprovação
da
proposta
de
emenda
à
constituição
pec
que
amplia
e
cria
novos
benefícios
sociais
a
três
meses
das
eleições
o
ministro
da
economia
paulo
guedes
chamou
nesta
quinta
14
os
críticos
das
medidas
de
“politicamente
enviesados”
e


bilionário
e
fundador
da
microsoft
bill
gates
criticou
as
criptomoedas
e
nfts
ele
afirmou
que
os
ativos
são
“100
baseados
na
teoria
do
maior
tolo”
bitcoin
cai
para
nova
mínima
de
18
meses
colapso
das
criptomoedas
se
aprofunda
para
ele
o
setor
cerimônia de toque de campainha para celebrar ações da eletrobras será amanhã o ministério da economia informou nesta segundafeira 13 que será realizada nesta terçafeira 14 às 12 horas de brasília em são paulo a cerimônia de toque de campainha para celebrar a oferta subsequente de ações da eletrobras após a solenidade está prevista uma coletiva de imprensa com autoridades dos ministérios da economia minas e energia 
cerimônia
de
toque
de
campainha
para
celebrar
ações
da
eletrobras
será
amanhã
o
ministério
da
economia
informou
nesta
segundafeira
13
que
será
realizada
nesta
terçafeira
14
às
12
horas
de
brasília
em
são
paulo
a
cerimônia
de
toque
de
campainha
para
celebrar
a
oferta
subsequente
de
ações
da
eletrobras
após
a
solenidade
está
prevista
uma

até
2029
como
parte
do
processo
de
adesão
do
brasil
ao
código
de
condenado a um século de prisão líder da ndrangheta é extraditado para a itália o ministério da justiça e segurança pública informou que o italiano rocco morabito apontado como líder da máfia calabresa ‘ndrangheta’ e conhecido como ‘rei da cocaína em milão’ foi extraditado para a itália na manhã desta quartafeira 6 a medida foi executada em coordenação com a pasta e a polícia federal a entrega de morabito 
condenado
a
um
século
de
prisão
líder
da
ndrangheta
é
extraditado
para
a
itália
o
ministério
da
justiça
e
segurança
pública
informou
que
o
italiano
rocco
morabito
apontado
como
líder
da
máfia
calabresa
‘ndrangheta’
e
conhecido
como
‘rei
da
cocaína
em
milão’
foi
extraditado
para
a
itália
na
manhã
desta
quartafeira
6
a
medida
foi
executada
em
coordenação
com
a
pasta
e
a
polícia
federal
a
entrega
de
morabito
investigação da invasão ao capitólio mostrará fúria de 3 horas dos apoiadores de trump por patricia zengerle washing

dólar
fragilizado
no
exterior
ante
moedas
principais
e
emergentes
após
dados
fracos
de
emprego
da
adp
e
quedas
de
pmis
dos
eua
enquanto
os
pmis
de
brasil
e
china
vieram
melhores
que
o
esperado
dólar sobe com cautela sobre atividade global em meio à agenda fraca o mercado de câmbio volta a operar o dólar em alta alinhado à tendência internacional com preocupações persistentes com a economia global após dados de atividade fracos da china e dos estados unidos o petróleo se fortaleceu após ampliar mais cedo as perdas acentuadas das duas últimas sessões os retornos dos treasuries de dois e 
dólar
sobe
com
cautela
sobre
atividade
global
em
meio
à
agenda
fraca
o
mercado
de
câmbio
volta
a
operar
o
dólar
em
alta
alinhado
à
tendência
internacional
com
preocupações
persistentes
com
a
economia
global
após
dados
de
atividade
fracos
da
china
e
dos
estados
unidos
o
petróleo
se
fortaleceu
após
ampliar
mais
cedo
as
perdas
acentuadas
das
duas
últimas
sessões
os
retornos
dos
treasuries
de
dois
e
igc elev

impulso
do
setor
financeiro
e
ações
ligadas
ao
consumo
doméstico
em
meio
a
humor
favorável
ao
risco
nos
mercados
globais
a
ata
da
última
reunião
de
política
monetária
do
banco
central
também
movimentou
a
sessão
bolsas
dólar recua de olho em ny positivo e queda ante emergentes ligadas a commodities o dólar ajustase em baixa na manhã desta quartafeira 26 após subir ontem a moeda americana acompanha a tendência negativa ante outras divisas emergentes e ligadas a commodities como peso mexicano diante do apetite por risco que apoia alta dos futuros de nova york contudo o índice dxy que compara o dólar ante seis moedas 
dólar
recua
de
olho
em
ny
positivo
e
queda
ante
emergentes
ligadas
a
commodities
o
dólar
ajustase
em
baixa
na
manhã
desta
quartafeira
26
após
subir
ontem
a
moeda
americana
acompanha
a
tendência
negativa
ante
outras
divisas
emergentes
e
ligadas
a
commodities
como
peso
mexicano
diante
do
apetite
por
risco
que
apoia
alta
dos
futuros
de
nova
york
contudo
o
índice
dxy
que
compara


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações das notícias entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nas notícias. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas caterogias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por caterogia (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**